<a href="https://colab.research.google.com/github/geoffkylin/APS360-Project/blob/master/baseline_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! unzip randcropped_split_augmented_mtsd_hand_selected_dataset.zip -d '/root/datasets1'

In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score




In [2]:
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
import os
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [3]:
data_transform = transforms.Compose([transforms.RandomResizedCrop (224,scale = (1,1)), transforms.ToTensor()])

In [4]:
data_dir = '/root/datasets1/'
train_dir = os.path.join(data_dir, 'train/')
val_dir = os.path.join(data_dir, 'val/')
test_dir = os.path.join(data_dir, 'test/' )

In [5]:
train_data = ImageFolder(train_dir,transform = data_transform)
val_data = ImageFolder(val_dir,transform = data_transform)
test_data = ImageFolder(test_dir,transform = data_transform)

In [6]:
print(train_data.classes)

['complementary--both-directions--g1', 'complementary--chevron-left--g1', 'complementary--chevron-right--g1', 'complementary--obstacle-delineator--g1', 'complementary--obstacle-delineator--g2', 'complementary--tow-away-zone--g1', 'information--airport--g2', 'information--bike-route--g1', 'information--disabled-persons--g1', 'information--gas-station--g1', 'information--highway-exit--g1', 'information--hospital--g1', 'information--telephone--g1', 'regulatory--do-not-block-intersection--g1', 'regulatory--dual-lanes-go-straight-on-left--g1', 'regulatory--dual-lanes-go-straight-on-right--g1', 'regulatory--go-straight--g3', 'regulatory--go-straight-or-turn-left--g2', 'regulatory--keep-left--g2', 'regulatory--keep-right--g4', 'regulatory--maximum-speed-limit-100--g3', 'regulatory--maximum-speed-limit-25--g2', 'regulatory--maximum-speed-limit-30--g3', 'regulatory--maximum-speed-limit-35--g2', 'regulatory--maximum-speed-limit-40--g3', 'regulatory--maximum-speed-limit-45--g3', 'regulatory--maxi

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, shuffle = True)
train_x = []
train_y = []
counter = 0
for i, j in train_loader:
    if counter <=5000:
        train_x.append(i)
        train_y.append(j)
        counter = counter +1
print(train_y)
print(train_x[0])

In [8]:
test_loader = torch.utils.data.DataLoader(test_data, shuffle = True)
test_x = []
test_y = []
counter = 0
for i, j in test_loader:
    if counter <= 1000:
        test_x.append(i)
        test_y.append(j)
        counter = counter +1


In [9]:
print(train_x[0].shape)
print(train_y[0].shape)

torch.Size([1, 3, 224, 224])
torch.Size([1])


In [10]:
train_x = torch.stack(train_x)
train_y = torch.stack(train_y)
print(train_x.shape)
print(train_y.shape)


torch.Size([5001, 1, 3, 224, 224])
torch.Size([5001, 1])


In [11]:
nsample, x,y,z,c = train_x.shape
train_x = train_x.reshape(nsample, x*y*z*c)
print(train_x.shape)

torch.Size([5001, 150528])


In [12]:
test_x = torch.stack(test_x)
test_y = torch.stack(test_y)
print(test_x.shape)
print(test_y.shape)

torch.Size([1001, 1, 3, 224, 224])
torch.Size([1001, 1])


In [13]:
nsample, x,y,z,c = test_x.shape
test_x = test_x.reshape(nsample, x*y*z*c)
print(test_x.shape)

torch.Size([1001, 150528])


In [14]:
parameters = {'bootstrap': True,
              'min_samples_leaf': 3,
              'n_estimators': 50, 
              'min_samples_split': 10,
              'max_features': 'sqrt',
              'max_depth': 6,
              'max_leaf_nodes': None}

In [15]:
RF_model = RandomForestClassifier(**parameters)
RF_model.fit(train_x, train_y)
RF_predictions = RF_model.predict(test_x)
score = accuracy_score(test_y ,RF_predictions)
print(score)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


0.4435564435564436


In [ ]:
# Extract single tree
estimator = RF_model.estimators_[5]

In [ ]:
from sklearn.tree import export_graphviz

from sklearn.datasets import load_iris
iris = load_iris()
# Export as dot file
export_graphviz(estimator, 
                out_file='tree.dot', 
                
                rounded = True, proportion = False, 
                precision = 2, filled = True)

In [ ]:
# Convert to png
from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

0

In [ ]:
# Display in jupyter notebook
from IPython.display import Image
Image(filename = 'tree.png')